In [1]:
# ─── 0. Imports ───────────────────────────────────────────────────────────────
import numpy as np
import pandas as pd
import joblib

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# ─── 1. Load & One-Hot Encode ────────────────────────────────────────────────
file_path = r"C:\Users\ayush\OneDrive\Desktop\bphc\Year 3 Sem 2\DOP\Lab Work\Models\Outside Only\Final DataSet\MLP Keras Version FInal DataSet Ver1\Final DataSet Combined 95percent for Model Training.xlsx"
df = pd.read_excel(file_path)

# one-hot encode the four angle categories
df = pd.get_dummies(df, columns=['Angle'], prefix='Angle')

# ─── 2. Build feature & target arrays ────────────────────────────────────────
cont_cols = ['X','Y','Z','Normal_X','Normal_Y','Normal_Z']
cat_cols  = ['Angle_40','Angle_50','Angle_60','Angle_70']
features = cont_cols + cat_cols

X_raw = df[features].values.astype(float)
y_raw = df['Z Scan'].values.reshape(-1,1).astype(float)

# ─── 3. Scale continuous inputs & target ─────────────────────────────────────
scaler_X = StandardScaler()
X_cont = scaler_X.fit_transform(df[cont_cols])
X = np.hstack([X_cont, df[cat_cols].values])

scaler_y = StandardScaler()
y = scaler_y.fit_transform(y_raw)

# save scalers for later inverse-transform
joblib.dump(scaler_X, 'scaler_X_zscan.pkl')
joblib.dump(scaler_y, 'scaler_y_zscan.pkl')

# ─── 4. Train/Test Split ─────────────────────────────────────────────────────
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

# ─── 5. Define Keras Model ───────────────────────────────────────────────────
model = Sequential([
    Dense(64, input_shape=(X_train.shape[1],)),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.1),

    Dense(32),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.1),

    Dense(16),
    LeakyReLU(alpha=0.1),
    BatchNormalization(),
    Dropout(0.1),

    Dense(1, activation='linear')
])

model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

model.summary()

# ─── 6. Callbacks ─────────────────────────────────────────────────────────────
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=1e-6
    ),
    ModelCheckpoint(
        'best_deep_mlp_zscan.h5',
        monitor='val_loss',
        save_best_only=True
    )
]

# ─── 7. Train ─────────────────────────────────────────────────────────────────
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=200,
    batch_size=1024,
    callbacks=callbacks,
    verbose=2
)

# ─── 8. Evaluate on Test Set ─────────────────────────────────────────────────
y_pred_scaled = model.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_true = scaler_y.inverse_transform(y_test)

mse_test = mean_squared_error(y_true, y_pred)
r2_test  = r2_score(y_true, y_pred)
print(f"\nTest MSE: {mse_test:.4f}   Test R²: {r2_test:.4f}")

# ─── 9. Save Final Model ──────────────────────────────────────────────────────
model.save('final_deep_mlp_zscan_keras.h5')


C:\Users\ayush\anaconda3\envs\tf310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\ayush\anaconda3\envs\tf310\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 64)                  │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 32)                  │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 16)                  │              64 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,777 (14.75 KB)

 Trainable params: 3,553 (13.88 KB)

...

242/242 - 3s - 14ms/step - loss: 0.0122 - mae: 0.0825 - val_loss: 4.8582e-04 - val_mae: 0.0168 - learning_rate: 5.0000e-04
Epoch 30/200
242/242 - 3s - 11ms/step - loss: 0.0123 - mae: 0.0831 - val_loss: 6.4262e-04 - val_mae: 0.0183 - learning_rate: 5.0000e-04
Epoch 31/200
242/242 - 3s - 12ms/step - loss: 0.0120 - mae: 0.0816 - val_loss: 5.1321e-04 - val_mae: 0.0161 - learning_rate: 2.5000e-04
Epoch 32/200


242/242 - 3s - 11ms/step - loss: 0.0121 - mae: 0.0820 - val_loss: 3.1091e-04 - val_mae: 0.0131 - learning_rate: 2.5000e-04
Epoch 33/200
242/242 - 7s - 27ms/step - loss: 0.0119 - mae: 0.0815 - val_loss: 4.5569e-04 - val_mae: 0.0158 - learning_rate: 2.5000e-04
Epoch 34/200
242/242 - 4s - 18ms/step - loss: 0.0120 - mae: 0.0817 - val_loss: 3.6764e-04 - val_mae: 0.0137 - learning_rate: 2.5000e-04
Epoch 35/200
242/242 - 4s - 16ms/step - loss: 0.0118 - mae: 0.0806 - val_loss: 7.2768e-04 - val_mae: 0.0198 - learning_rate: 2.5000e-04
Epoch 36/200
242/242 - 3s - 11ms/step - loss: 0.0117 - mae: 0.0806 - val_loss: 4.7362e-04 - val_mae: 0.0161 - learning_rate: 2.5000e-04
Epoch 37/200
242/242 - 3s - 11ms/step - loss: 0.0118 - mae: 0.0810 - val_loss: 9.1385e-04 - val_mae: 0.0236 - learning_rate: 2.5000e-04
Epoch 38/200
242/242 - 3s - 12ms/step - loss: 0.0118 - mae: 0.0811 - val_loss: 3.1596e-04 - val_mae: 0.0131 - learning_rate: 2.5000e-04
Epoch 39/200
242/242 - 3s - 13ms/step - loss: 0.0116 - mae: 0

242/242 - 3s - 12ms/step - loss: 0.0113 - mae: 0.0790 - val_loss: 3.0942e-04 - val_mae: 0.0127 - learning_rate: 1.2500e-04
Epoch 45/200
242/242 - 3s - 12ms/step - loss: 0.0111 - mae: 0.0784 - val_loss: 3.6423e-04 - val_mae: 0.0139 - learning_rate: 1.2500e-04
Epoch 46/200
242/242 - 3s - 11ms/step - loss: 0.0113 - mae: 0.0792 - val_loss: 5.1352e-04 - val_mae: 0.0165 - learning_rate: 1.2500e-04
Epoch 47/200
242/242 - 3s - 14ms/step - loss: 0.0113 - mae: 0.0791 - val_loss: 3.2671e-04 - val_mae: 0.0130 - learning_rate: 1.2500e-04
Epoch 48/200
242/242 - 3s - 10ms/step - loss: 0.0114 - mae: 0.0797 - val_loss: 3.2035e-04 - val_mae: 0.0127 - learning_rate: 1.2500e-04
Epoch 49/200
242/242 - 4s - 15ms/step - loss: 0.0112 - mae: 0.0788 - val_loss: 3.9690e-04 - val_mae: 0.0146 - learning_rate: 1.2500e-04
Epoch 50/200


242/242 - 3s - 12ms/step - loss: 0.0112 - mae: 0.0785 - val_loss: 2.6215e-04 - val_mae: 0.0117 - learning_rate: 1.2500e-04
Epoch 51/200
242/242 - 3s - 12ms/step - loss: 0.0109 - mae: 0.0772 - val_loss: 4.1005e-04 - val_mae: 0.0142 - learning_rate: 1.2500e-04
Epoch 52/200
242/242 - 3s - 12ms/step - loss: 0.0114 - mae: 0.0799 - val_loss: 2.7730e-04 - val_mae: 0.0122 - learning_rate: 1.2500e-04
Epoch 53/200
242/242 - 3s - 11ms/step - loss: 0.0113 - mae: 0.0792 - val_loss: 3.3648e-04 - val_mae: 0.0128 - learning_rate: 6.2500e-05
Epoch 54/200
242/242 - 3s - 11ms/step - loss: 0.0112 - mae: 0.0786 - val_loss: 4.3144e-04 - val_mae: 0.0154 - learning_rate: 6.2500e-05
Epoch 55/200
242/242 - 3s - 12ms/step - loss: 0.0108 - mae: 0.0771 - val_loss: 4.1947e-04 - val_mae: 0.0148 - learning_rate: 6.2500e-05
Epoch 56/200
242/242 - 2s - 10ms/step - loss: 0.0112 - mae: 0.0790 - val_loss: 3.9302e-04 - val_mae: 0.0140 - learning_rate: 6.2500e-05
Epoch 57/200
242/242 - 3s - 11ms/step - loss: 0.0114 - mae: 0


Test MSE: 0.1235   Test R²: 0.9997


In [3]:
import numpy as np
import pandas as pd
import joblib
from tensorflow.keras.models import load_model

# ─── File paths ───────────────────────────────────────────────────────────────
excel1_path = r"C:\Users\ayush\OneDrive\Desktop\bphc\Year 3 Sem 2\DOP\Lab Work\Models\Outside Only\Final DataSet\MLP Keras Version FInal DataSet Ver1\Final DataSet Cross Validation 5percent Data without Z scan.xlsx"
excel2_path = r"C:\Users\ayush\OneDrive\Desktop\bphc\Year 3 Sem 2\DOP\Lab Work\Models\Outside Only\Final DataSet\MLP Keras Version FInal DataSet Ver1\Final DataSet Cross Validation 5percent Data with Z scan.xlsx"
excel3_path = r"C:\Users\ayush\OneDrive\Desktop\bphc\Year 3 Sem 2\DOP\Lab Work\Models\Outside Only\Final DataSet\MLP Keras Version FInal DataSet Ver1\MLP Keras Ver1 Results.xlsx"

model_path    = "final_deep_mlp_zscan_keras.h5"  # or your best_deep_mlp_zscan.h5
scalerX_path  = "scaler_X_zscan.pkl"
scalerY_path  = "scaler_y_zscan.pkl"

# ─── Load model & scalers ─────────────────────────────────────────────────────
from tensorflow.keras.models import load_model
model = load_model(model_path, compile=False)

scaler_X = joblib.load(scalerX_path)
scaler_y = joblib.load(scalerY_path)

# ─── Read the two Excels ──────────────────────────────────────────────────────
df1 = pd.read_excel(excel1_path)
df2 = pd.read_excel(excel2_path)

# ─── Prepare features exactly as in training ─────────────────────────────────
cont_cols = ['X','Y','Z','Normal_X','Normal_Y','Normal_Z']
cat_cols  = ['Angle_40','Angle_50','Angle_60','Angle_70']

# 1) One-hot encode the Angle column
df_feat = pd.get_dummies(df1, columns=['Angle'], prefix='Angle')

# 2) Ensure all four Angle_[…] columns are present
for c in cat_cols:
    if c not in df_feat:
        df_feat[c] = 0

# 3) Scale continuous features
X_cont = scaler_X.transform(df_feat[cont_cols])

# 4) Stack one-hot columns
X_cat  = df_feat[cat_cols].values
X      = np.hstack([X_cont, X_cat])

# ─── Predict & inverse-scale ──────────────────────────────────────────────────
y_pred_scaled = model.predict(X)                  # shape (n,1)
y_pred        = scaler_y.inverse_transform(y_pred_scaled).flatten()

# ─── Build result DataFrame ──────────────────────────────────────────────────
df_out = df1.copy()
df_out['Z Scan (Predicted)'] = y_pred
df_out['Delta Z (Predicted)'] = df_out['Z'] - df_out['Z Scan (Predicted)']

# bring in the true values
df_out['Z Scan (True)']   = df2['Z Scan'].values
df_out['Delta Z (True)']  = df2['Delta Z'].values

# error between true vs pred
df_out['Z Scan Error']    = df_out['Z Scan (True)'] - df_out['Z Scan (Predicted)']

# ─── Save to new Excel ────────────────────────────────────────────────────────
df_out.to_excel(excel3_path, index=False)
print(f"✅ Done — comparison saved to:\n   {excel3_path}")


510/510 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
✅ Done — comparison saved to:
   C:\Users\ayush\OneDrive\Desktop\bphc\Year 3 Sem 2\DOP\Lab Work\Models\Outside Only\Final DataSet\MLP Keras Version FInal DataSet Ver1\MLP Keras Ver1 Results.xlsx


In [4]:
# ─── 1. Imports ────────────────────────────────────────────────────────────────
import numpy as np
import pandas as pd
import joblib
from tensorflow.keras.models import load_model

# ─── 2. Prompt for file paths ─────────────────────────────────────────────────
input_path  = r"C:\Users\ayush\OneDrive\Desktop\bphc\Year 3 Sem 2\DOP\Lab Work\Models\Outside Only\Final DataSet\MLP Keras Version FInal DataSet Ver1\Cone 65\Cone 65 Outer Vertices and Normal Coordinates.xlsx"
output_path = r"C:\Users\ayush\OneDrive\Desktop\bphc\Year 3 Sem 2\DOP\Lab Work\Models\Outside Only\Final DataSet\MLP Keras Version FInal DataSet Ver1\Cone 65\Cone 65 Keras Result.xlsx"

# ─── 3. Load your trained Keras model & scalers ───────────────────────────────
#   (change these names if yours differ)
model       = load_model('final_deep_mlp_zscan_keras.h5', compile=False)
scaler_X    = joblib.load('scaler_X_zscan.pkl')
scaler_y    = joblib.load('scaler_y_zscan.pkl')

# ─── 4. Read the input sheet ───────────────────────────────────────────────────
df = pd.read_excel(input_path)

# ─── 5. One-hot encode Angle exactly as in training ────────────────────────────
# training had four dummies: Angle_40,50,60,70
cat_cols = ['Angle_40','Angle_50','Angle_60','Angle_70']
df_enc   = pd.get_dummies(df, columns=['Angle'], prefix='Angle')

# ensure all four columns exist (any new angle → all zeros)
for c in cat_cols:
    if c not in df_enc:
        df_enc[c] = 0

# ─── 6. Scale & assemble your feature matrix ─────────────────────────────────
cont_cols = ['X','Y','Z','Normal_X','Normal_Y','Normal_Z']

X_cont = scaler_X.transform(df_enc[cont_cols])
X_cat  = df_enc[cat_cols].values
X_all  = np.hstack([X_cont, X_cat])

# ─── 7. Predict & invert the y-scale ─────────────────────────────────────────
y_pred_scaled = model.predict(X_all)
y_pred         = scaler_y.inverse_transform(y_pred_scaled).flatten()

# ─── 8. Append the predictions & save ─────────────────────────────────────────
df['Z Scan (Predicted)'] = y_pred
df.to_excel(output_path, index=False)

print(f"✅ Done!  Predictions written to:\n   {output_path}")


2976/2976 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step
✅ Done!  Predictions written to:
   C:\Users\ayush\OneDrive\Desktop\bphc\Year 3 Sem 2\DOP\Lab Work\Models\Outside Only\Final DataSet\MLP Keras Version FInal DataSet Ver1\Cone 65\Cone 65 Keras Result.xlsx


In [12]:
import numpy as np
import joblib
from tensorflow.keras.models import load_model

# 1) Load model & scalers
model = load_model('final_deep_mlp_zscan_keras.h5', compile=False)
scaler_X = joblib.load('scaler_X_zscan.pkl')
scaler_y = joblib.load('scaler_y_zscan.pkl')

# 2) Define possible angles for one‐hot encoding
angles = [40, 50, 60, 70]

# 3) Prompt user for inputs
x = float(input("Enter X: "))
y = float(input("Enter Y: "))
z = float(input("Enter Z: "))
nx = float(input("Enter Normal_X: "))
ny = float(input("Enter Normal_Y: "))
nz = float(input("Enter Normal_Z: "))
angle_input = int(input("Enter Angle (40, 50, 60, or 70): "))

# 4) Build and scale the continuous part
X_cont = np.array([[x, y, z, nx, ny, nz]])         # shape (1,6)
X_scaled = scaler_X.transform(X_cont)              # shape (1,6)

# 5) Create one‐hot encoding and shape it as a row
ohe = np.array([[1 if angle_input == a else 0 for a in angles]])  # shape (1,4)

# 6) Concatenate to form the final feature vector
X_final = np.hstack([X_scaled, ohe])               # shape (1,10)

# 7) Predict and invert the target scaling
y_pred_scaled = model.predict(X_final)             # shape (1,1)
y_pred = scaler_y.inverse_transform(y_pred_scaled)[0,0]

# 8) Display result
print(f"\nPredicted Z Scan = {y_pred:.4f}")


Enter X:  1
Enter Y:  1
Enter Z:  11
Enter Normal_X:  1
Enter Normal_Y:  1
Enter Normal_Z:  1
Enter Angle (40, 50, 60, or 70):  23


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step

Predicted Z Scan = 4.0171


C:\Users\ayush\anaconda3\envs\tf310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
